# Deep Learning Challenge - Training
## Michelle Rohrer

Dieses Notebook enthält alle Trainings.

### Pakete laden

In [3]:
import warnings
warnings.filterwarnings('ignore', category=UserWarning, module='pydantic')

# Dann der Rest Ihrer Imports
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import wandb
from dotenv import load_dotenv
import pickle

import torch
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms
import torch.nn as nn
import torch.nn.functional as F
from torchsummary import summary

from src.plots import (
    plot_training_curves
)
from src.model import BaselineCNN
from src.test_train import (
    overfitting_test_batch,
    train_model,
    hyperparameter_tuning_with_wandb
)
from src.evaluation import (
    cross_validation_training
)

load_dotenv()
wandb_key = os.getenv('KEY')

if wandb_key:
    print("WanDB Key erfolgreich geladen")
else:
    print("WanDB Key nicht gefunden")


WanDB Key erfolgreich geladen


### Daten laden


In [4]:
# .pkl laden
with open('data/train_dataset.pkl', 'rb') as f:
    train_dataset = pickle.load(f)

with open('data/val_dataset.pkl', 'rb') as f:
    val_dataset = pickle.load(f)

with open('data/test_dataset.pkl', 'rb') as f:
    test_dataset = pickle.load(f)


### Training des Basismodells

#### Weights & Biases (wandb) Integration

**Zweck:** Automatisches Tracking aller Experimente für bessere Reproduzierbarkeit und Analyse.

**Was wird getrackt:**
- **Hyperparameter:** Lernrate, Batch-Größe, Optimizer-Einstellungen
- **Metriken:** Training/Validation Loss und Accuracy pro Epoche
- **Test-Metriken:** Accuracy, Precision, Recall, F1-Score, Top-K Accuracy
- **Konfusionsmatrix:** Visualisierung der Klassifikationsergebnisse
- **Lernkurven:** Automatische Plots für alle Konfigurationen


In [ ]:
# wandb Setup
# wandb API Key aus .env laden
wandb_key = os.getenv('KEY')
if wandb_key:
    wandb.login(key=wandb_key)
    print("wandb erfolgreich authentifiziert")
else:
    print("wandb API Key nicht in .env gefunden")

# Device Setup
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
print(f"Verwende Device: {device}")

# Klassennamen für Evaluation
class_names = list(full_train_dataset.class_to_idx.keys())
print(f"Klassen: {class_names}")


In [ ]:
# Basismodell erstellen und trainieren
print("=== Training des Basismodells ===")
print("Konfiguration: SGD (momentum=0), LR=0.01, Batch=64, 200 Epochen")

# Modell initialisieren
model = BaselineCNN(img_size=img_size, num_classes=num_classes).to(device)

# Training durchführen mit wandb
train_losses, val_losses, train_accs, val_accs = train_model(
    model=model,
    device=device,
    train_loader=train_loader,
    val_loader=val_loader,
    num_epochs=200,
    learning_rate=0.01,
    batch_size=64,
    use_wandb=True,
    run_name="baseline_model",
    early_stopping=True,
    patience=15,  # Stoppe wenn 15 Epochen ohne Verbesserung
    min_delta=0.001
)

# Lernkurven plotten
plot_training_curves(train_losses, val_losses, train_accs, val_accs, 
                    title="Basismodell: SGD (momentum=0), LR=0.01, Batch=64")

# Modell speichern
torch.save(model.state_dict(), 'models/baseline_model.pth')
print("Basismodell gespeichert als 'models/baseline_model.pth'")


### Hyperparameter-Tuning

**Ziel:** Vergleich verschiedener Lernraten und Batch-Größen anhand der Lernkurven und Metriken.

**Testkonfigurationen:**
- **Lernraten:** [0.001, 0.01, 0.1]
- **Batch-Größen:** [32, 64, 128]
- **Epochen:** 100 (für schnelleren Vergleich)
- **Optimizer:** SGD ohne Momentum


In [ ]:
# Hyperparameter-Tuning mit wandb Integration
print("=== Hyperparameter-Tuning mit wandb ===")

# Testkonfigurationen
learning_rates = [0.001, 0.01, 0.1]
batch_sizes = [16, 32, 64]  # Kleinere Batches für bessere Performance
num_epochs = 50  # Weniger Epochen für schnellere Tests

# Hyperparameter-Tuning mit wandb durchführen
results = hyperparameter_tuning_with_wandb(
    train_dataset=train_dataset,
    val_dataset=val_dataset,
    learning_rates=learning_rates,
    batch_sizes=batch_sizes,
    num_epochs=num_epochs
)

print("\n=== Hyperparameter-Tuning abgeschlossen ===")
print("Alle Experimente wurden in wandb gespeichert!")


In [ ]:
# Visualisierung der Hyperparameter-Ergebnisse
print("=== Hyperparameter-Vergleich ===")

# Ergebnisse als DataFrame für bessere Übersicht
import pandas as pd

# Zusammenfassung der Ergebnisse
summary_data = []
for config_key, result in results.items():
    summary_data.append({
        'Learning Rate': result['learning_rate'],
        'Batch Size': result['batch_size'],
        'Final Train Acc (%)': result['final_train_acc'],
        'Final Val Acc (%)': result['final_val_acc'],
        'Final Train Loss': result['final_train_loss'],
        'Final Val Loss': result['final_val_loss']
    })

df_summary = pd.DataFrame(summary_data)
df_summary = df_summary.sort_values('Final Val Acc (%)', ascending=False)

print("Ergebnisse sortiert nach Validation Accuracy:")
print(df_summary.to_string(index=False))

# Beste Konfiguration finden
best_config = df_summary.iloc[0]
print(f"\nBeste Konfiguration:")
print(f"Learning Rate: {best_config['Learning Rate']}")
print(f"Batch Size: {best_config['Batch Size']}")
print(f"Validation Accuracy: {best_config['Final Val Acc (%)']:.2f}%")


In [ ]:
# Lernkurven-Vergleich für verschiedene Konfigurationen
fig, axes = plt.subplots(2, 3, figsize=(18, 12))
axes = axes.flatten()

# Für jede Lernrate einen Subplot
for i, lr in enumerate(learning_rates):
    ax = axes[i]
    
    for batch_size in batch_sizes:
        config_key = f"LR_{lr}_Batch_{batch_size}"
        result = results[config_key]
        
        epochs = range(1, len(result['val_accs']) + 1)
        ax.plot(epochs, result['val_accs'], 
               label=f'Batch {batch_size}', 
               linewidth=2, marker='o', markersize=4)
    
    ax.set_title(f'Learning Rate = {lr}')
    ax.set_xlabel('Epoch')
    ax.set_ylabel('Validation Accuracy (%)')
    ax.legend()
    ax.grid(True, alpha=0.3)

# Für jede Batch-Größe einen Subplot
for i, batch_size in enumerate(batch_sizes):
    ax = axes[i + 3]
    
    for lr in learning_rates:
        config_key = f"LR_{lr}_Batch_{batch_size}"
        result = results[config_key]
        
        epochs = range(1, len(result['val_accs']) + 1)
        ax.plot(epochs, result['val_accs'], 
               label=f'LR {lr}', 
               linewidth=2, marker='o', markersize=4)
    
    ax.set_title(f'Batch Size = {batch_size}')
    ax.set_xlabel('Epoch')
    ax.set_ylabel('Validation Accuracy (%)')
    ax.legend()
    ax.grid(True, alpha=0.3)

plt.suptitle('Hyperparameter-Tuning: Lernkurven-Vergleich', fontsize=16)
plt.tight_layout()
plt.show()


### Cross-Validation für statistische Fehlerschätzung

**Ziel:** Schätzung des statistischen Fehlers der Metriken durch Cross-Validation.

**Konfiguration:**
- **5-Fold Cross-Validation** auf dem Trainingsdatensatz
- **Beste Hyperparameter** aus dem Tuning verwenden
- **Statistische Auswertung:** Mittelwert ± Standardabweichung


In [ ]:
# Cross-Validation für statistische Fehlerschätzung
print("=== Cross-Validation ===")

# Beste Hyperparameter aus dem Tuning verwenden
best_lr = best_config['Learning Rate']
best_batch = int(best_config['Batch Size'])

print(f"Verwende beste Konfiguration: LR={best_lr}, Batch={best_batch}")

# Cross-Validation durchführen
cv_results = cross_validation_training(
    train_dataset=full_train_dataset,
    num_folds=5,
    num_epochs=50, 
    learning_rate=best_lr,
    batch_size=best_batch
)

print(f"\n=== Cross-Validation Ergebnisse ===")
print(f"Training Accuracy: {cv_results['train_accuracies']['mean']:.4f} ± {cv_results['train_accuracies']['std']:.4f}")
print(f"Validation Accuracy: {cv_results['val_accuracies']['mean']:.4f} ± {cv_results['val_accuracies']['std']:.4f}")
print(f"Training Loss: {cv_results['train_losses']['mean']:.4f} ± {cv_results['train_losses']['std']:.4f}")
print(f"Validation Loss: {cv_results['val_losses']['mean']:.4f} ± {cv_results['val_losses']['std']:.4f}")

# Visualisierung der CV-Ergebnisse
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Accuracy Boxplot
acc_data = [cv_results['train_accuracies']['values'], cv_results['val_accuracies']['values']]
axes[0].boxplot(acc_data, labels=['Training', 'Validation'])
axes[0].set_title('Cross-Validation: Accuracy Distribution')
axes[0].set_ylabel('Accuracy (%)')
axes[0].grid(True, alpha=0.3)

# Loss Boxplot
loss_data = [cv_results['train_losses']['values'], cv_results['val_losses']['values']]
axes[1].boxplot(loss_data, labels=['Training', 'Validation'])
axes[1].set_title('Cross-Validation: Loss Distribution')
axes[1].set_ylabel('Loss')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Ergebnisse speichern
import json
with open('results/cross_validation_results.json', 'w') as f:
    json.dump({
        'best_hyperparameters': {
            'learning_rate': best_lr,
            'batch_size': best_batch
        },
        'cv_results': cv_results
    }, f, indent=2)

print("Cross-Validation Ergebnisse gespeichert als 'results/cross_validation_results.json'")
